In [3]:
from cbrain.imports import *
from cbrain.data_generator import *
from cbrain.cam_constants import *
from cbrain.losses import *
from cbrain.utils import limit_mem
from cbrain.layers import *
from cbrain.data_generator import DataGenerator
import tensorflow as tf
from tensorflow import math as tfm
#import tensorflow_probability as tfp
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
import xarray as xr
import numpy as np
from cbrain.model_diagnostics import ModelDiagnostics
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as imag
import scipy.integrate as sin
#import cartopy.crs as ccrs
import matplotlib.ticker as mticker
#from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import pickle
import sklearn
from sklearn.linear_model import LinearRegression
from scipy import stats
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns
from cbrain.imports import *
from cbrain.utils import *
from cbrain.normalization import *
import h5py
from sklearn.preprocessing import OneHotEncoder
from cbrain.climate_invariant import *
import yaml
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [4]:
# Load coordinates (just pick any file from the climate model run)
coor = xr.open_dataset("/DFS-L/DATA/pritchard/ankitesg/data/CESM2_f19_v13_updated_NN_pelayout01_ens_07.cam.h1.2003-01-22-00000.nc",\
                    decode_times=False)
lat = coor.lat; lon = coor.lon; lev = coor.lev;
DATA_DIR = '/DFS-L/DATA/pritchard/ankitesg/datav3/'
hyam = coor.hyam
hybm = coor.hybm
scale_dict = load_pickle('/export/nfs0home/ankitesg/tom/CBRAIN-CAM/nn_config/scale_dicts/2020_10_16_scale_dict_RG.pkl')['scale_dict_RG']

In [5]:
lev

<xarray.DataArray 'lev' (lev: 26)>
array([  3.544638,   7.388814,  13.967214,  23.944625,  37.23029 ,  53.114605,
        70.05915 ,  85.439115, 100.514695, 118.250335, 139.115395, 163.66207 ,
       192.539935, 226.513265, 266.481155, 313.501265, 368.81798 , 433.895225,
       510.455255, 600.5242  , 696.79629 , 787.70206 , 867.16076 , 929.648875,
       970.55483 , 992.5561  ])
Coordinates:
  * lev      (lev) float64 3.545 7.389 13.97 23.94 ... 867.2 929.6 970.6 992.6
Attributes:
    long_name:      hybrid level at midpoints (1000*(A+B))
    units:          hPa
    positive:       down
    standard_name:  atmosphere_hybrid_sigma_pressure_coordinate
    formula_terms:  a: hyam b: hybm p0: P0 ps: PS

In [6]:
class DataGeneratorClimInvRealGeo(DataGenerator):

    def __init__(self, data_fn, input_vars, output_vars,
             norm_fn=None, input_transform=None, output_transform=None,
             batch_size=1024, shuffle=True, xarray=False, var_cut_off=None, normalize_flag=True,
             rh_trans=True,t2tns_trans=True,
             lhflx_trans=True,
             scaling=True,interpolate=True,
             hyam=None,hybm=None,
             inp_subRH=None,inp_divRH=None,
             inp_subTNS=None,inp_divTNS=None,
             lev=None, interm_size=40,
             lower_lim=6,
             is_continous=True,Tnot=5,
                mode='train', exp=None):
        self.scaling = scaling
        self.interpolate = interpolate
        self.rh_trans = rh_trans
        self.t2tns_trans = t2tns_trans
        self.lhflx_trans = lhflx_trans
        self.inp_shape = 64
        self.exp = exp
        self.mode=mode
        super().__init__(data_fn, input_vars,output_vars,norm_fn,input_transform,output_transform,
                        batch_size,shuffle,xarray,var_cut_off,normalize_flag) ## call the base data generator
        self.inp_sub = self.input_transform.sub
        self.inp_div = self.input_transform.div
        self.new_idx = np.concatenate((self.input_idxs[8:26],self.input_idxs[34:52],self.input_idxs[60:78],self.input_idxs[86:104],\
        self.input_idxs[104:]))
        self.new_idx = np.concatenate((np.arange(8,26),np.arange(34,52),np.arange(60,78),np.arange(86,104),np.arange(104,108)))


    def __getitem__(self, index):
        # Compute start and end indices for batch
        start_idx = index * self.batch_size
        end_idx = start_idx + self.batch_size

        # Grab batch from data
        batch = self.data_ds['vars'][start_idx:end_idx]
#         print(self.new_idx)
        # Split into inputs and outputs
        X = batch[:, self.input_idxs]
        Y = batch[:, self.output_idxs]
        # Normalize
        X_norm = self.input_transform.transform(X)
        Y = self.output_transform.transform(Y)
        return X_norm[:,self.new_idx], Y

In [7]:
in_vars = ['QBP','TBP','CLDLIQBP','CLDICEBP','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars = ['QBCTEND','TBCTEND','CLDLIQBCTEND', 'CLDICEBCTEND', 'NN2L_FLWDS', 'NN2L_PRECC', 
            'NN2L_PRECSC', 'NN2L_SOLL', 'NN2L_SOLLD', 'NN2L_SOLS', 'NN2L_SOLSD', 'NN2L_NETSW']

In [8]:
TRAINFILE = 'RG_SP_M4K_train_shuffle.nc'
NORMFILE = 'RG_SP_M4K_NORM_norm.nc'
VALIDFILE = 'RG_SP_M4K_valid.nc'

In [90]:
train_gen_bf = DataGeneratorClimInvRealGeo(
    data_fn = f'{DATA_DIR}{TRAINFILE}',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = f'{DATA_DIR}{NORMFILE}',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True,
    normalize_flag=True,
    lev=lev,
    hyam=hyam,hybm=hybm,
    rh_trans = False,t2tns_trans=False,
    lhflx_trans=False,
    scaling=False,
    interpolate=False
)


In [91]:
train_gen_bf[0][0].shape

(1024, 76)

In [92]:
valid_gen_bf = DataGeneratorClimInvRealGeo(
    data_fn = f'{DATA_DIR}{VALIDFILE}',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = f'{DATA_DIR}{NORMFILE}',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True,
    normalize_flag=True,
    lev=lev,
    hyam=hyam,hybm=hybm,
    rh_trans = False,t2tns_trans=False,
    lhflx_trans=False,
    scaling=False,
    interpolate=False
)


In [93]:
valid_gen_bf[0][0].shape

(1024, 76)

In [94]:
model = Sequential()
model.add(Input(shape=(76,)))
model.add(Dense(128, activation='linear'))
model.add(LeakyReLU(alpha=0.3))
for i in range (4):
    model.add(Dense(128, activation='linear'))
    model.add(LeakyReLU(alpha=0.3))
    
for i in range (3):
    model.add(Dense(256, activation='linear'))
    model.add(LeakyReLU(alpha=0.3))
    
model.add(Dense(112, activation='linear'))

In [95]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 128)               9856      
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 128)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 128)              

In [96]:
opt = tf.keras.optimizers.Adam()

In [97]:
model.compile(optimizer=opt, loss='mse')

In [98]:
# model.compile(tf.keras.optimizers.Adam(), loss="mse")
path_HDF5 = '/DFS-L/DATA/pritchard/ankitesg/models/'
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint(path_HDF5+'BF_RG_TrimV1.h5',save_best_only=True, monitor='val_loss', mode='min')

In [99]:
with tf.device('/gpu:0'):
    Nep = 15
    model.fit_generator(train_gen_bf, epochs=Nep, validation_data=valid_gen_bf,\
                  callbacks=[earlyStopping, mcp_save])

Epoch 1/15
42118/42120 [============================>.] - ETA: 0s - batch: 21058.5000 - size: 1024.0000 - loss: 225.7975WARNING:tensorflow:From /export/home/ankitesg/anaconda3/envs/CBrainEnv2/lib/python3.6/site-packages/tensorflow/python/keras/engine/training_v1.py:2048: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


42120/42120 [==============================] - 444s 11ms/step - batch: 21059.5000 - size: 1024.0000 - loss: 225.7952 - val_loss: 196.1439
Epoch 2/15
42120/42120 [==============================] - 397s 9ms/step - batch: 21059.5000 - size: 1024.0000 - loss: 189.1131 - val_loss: 184.3258
Epoch 3/15
42120/42120 [==============================] - 732s 17ms/step - batch: 21059.5000 - size: 1024.0000 - loss: 181.9647 - val_loss: 182.0263
Epoch 4/15
42120/42120 [==============================] - 684s 16ms/step - batch: 21059.5000 - size: 1024.0000 - loss: 178.1164 - val_loss: 184.1667
Epoch 5/15
42120/42120 [==============================] - 382s 9ms/step - batch: 21059.5000 - size: 1024.0000 - loss: 175.4785 - val_loss: 175.8900
Epoch 6/15
42120/42120 [==============================] - 655s 16ms/step - batch: 21059.5000 - size: 1024.0000 - loss: 173.5932 - val_loss: 177.5319
Epoch 7/15
42120/42120 [==============================] - 379s 9ms/step - batch: 21059.5000 - size: 1024.0000 - loss: 1

### Trimmed Output

In [15]:
class DataGeneratorClimInvRealGeo(DataGenerator):

    def __init__(self, data_fn, input_vars, output_vars,
             norm_fn=None, input_transform=None, output_transform=None,
             batch_size=1024, shuffle=True, xarray=False, var_cut_off=None, normalize_flag=True,
             rh_trans=True,t2tns_trans=True,
             lhflx_trans=True,
             scaling=True,interpolate=True,
             hyam=None,hybm=None,
             inp_subRH=None,inp_divRH=None,
             inp_subTNS=None,inp_divTNS=None,
             lev=None, interm_size=40,
             lower_lim=6,
             is_continous=True,Tnot=5,
                mode='train', exp=None):
        self.scaling = scaling
        self.interpolate = interpolate
        self.rh_trans = rh_trans
        self.t2tns_trans = t2tns_trans
        self.lhflx_trans = lhflx_trans
        self.inp_shape = 64
        self.exp = exp
        self.mode=mode
        super().__init__(data_fn, input_vars,output_vars,norm_fn,input_transform,output_transform,
                        batch_size,shuffle,xarray,var_cut_off,normalize_flag) ## call the base data generator
        self.inp_sub = self.input_transform.sub
        self.inp_div = self.input_transform.div
        self.new_idx = np.concatenate((np.arange(8,26),np.arange(34,52),np.arange(60,78),np.arange(86,104),np.arange(104,108)))
        self.new_output_idx = np.concatenate((np.arange(8,26),np.arange(26,52),np.arange(60,78),np.arange(86,104),np.arange(104,112)))

    def __getitem__(self, index):
        # Compute start and end indices for batch
        start_idx = index * self.batch_size
        end_idx = start_idx + self.batch_size
        # Grab batch from data
        batch = self.data_ds['vars'][start_idx:end_idx]
        X = batch[:, self.input_idxs]
        Y = batch[:, self.output_idxs]
        # Normalize
        X_norm = self.input_transform.transform(X)
        Y = self.output_transform.transform(Y)
        return X_norm[:,self.new_idx], Y[:,self.new_output_idx]

In [16]:
in_vars = ['QBP','TBP','CLDLIQBP','CLDICEBP','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars = ['QBCTEND','TBCTEND','CLDLIQBCTEND', 'CLDICEBCTEND', 'NN2L_FLWDS', 'NN2L_PRECC', 
            'NN2L_PRECSC', 'NN2L_SOLL', 'NN2L_SOLLD', 'NN2L_SOLS', 'NN2L_SOLSD', 'NN2L_NETSW']

In [17]:
TRAINFILE = 'RG_SP_M4K_train_shuffle.nc'
NORMFILE = 'RG_SP_M4K_NORM_norm.nc'
VALIDFILE = 'RG_SP_M4K_valid.nc'

In [18]:
train_gen_bf = DataGeneratorClimInvRealGeo(
    data_fn = f'{DATA_DIR}{TRAINFILE}',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = f'{DATA_DIR}{NORMFILE}',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True,
    normalize_flag=True,
    lev=lev,
    hyam=hyam,hybm=hybm,
    rh_trans = False,t2tns_trans=False,
    lhflx_trans=False,
    scaling=False,
    interpolate=False
)


In [20]:
train_gen_bf[0][1].shape

(1024, 88)

In [21]:
valid_gen_bf = DataGeneratorClimInvRealGeo(
    data_fn = f'{DATA_DIR}{VALIDFILE}',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = f'{DATA_DIR}{NORMFILE}',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True,
    normalize_flag=True,
    lev=lev,
    hyam=hyam,hybm=hybm,
    rh_trans = False,t2tns_trans=False,
    lhflx_trans=False,
    scaling=False,
    interpolate=False
)


In [22]:
model = Sequential()
model.add(Input(shape=(76,)))
model.add(Dense(128, activation='linear'))
model.add(LeakyReLU(alpha=0.3))
for i in range (4):
    model.add(Dense(128, activation='linear'))
    model.add(LeakyReLU(alpha=0.3))
    
for i in range (3):
    model.add(Dense(256, activation='linear'))
    model.add(LeakyReLU(alpha=0.3))
    
model.add(Dense(88, activation='linear'))

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               9856      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               1

In [24]:
opt = tf.keras.optimizers.Adam()

In [25]:
model.compile(optimizer=opt, loss='mse')

In [26]:
# model.compile(tf.keras.optimizers.Adam(), loss="mse")
path_HDF5 = '/DFS-L/DATA/pritchard/ankitesg/models/'
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint(path_HDF5+'BF_RG_TrimV2.h5',save_best_only=True, monitor='val_loss', mode='min')

In [27]:
with tf.device('/gpu:0'):
    Nep = 15
    model.fit_generator(train_gen_bf, epochs=Nep, validation_data=valid_gen_bf,\
                  callbacks=[earlyStopping, mcp_save])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/15
42117/42120 [============================>.] - ETA: 0s - batch: 21058.0000 - size: 1024.0000 - loss: 287.5760WARNING:tensorflow:From /export/home/ankitesg/anaconda3/envs/CBrainEnv2/lib/python3.6/site-packages/tensorflow/python/keras/engine/training_v1.py:2048: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
42120/42120 [==============================] - 439s 10ms/step - batch: 21059.5000 - size: 1024.0000 - loss: 287.5732 - val_loss: 252.8144
Epoch 2/15
42120/42120 [==============================] - 439s 10ms/step - batch: 21059.5000 - size: 1024.0000 - loss: 241.1796 - val_loss: 242.6845
Epoch 3/15
42120/42120 [==============================] - 442s 10ms/step - batch: 21059.5000 - size: 1024.0000 - loss: 231.850

KeyboardInterrupt: 

### RH Trimmed

In [29]:
class DataGeneratorClimInvRealGeo(DataGenerator):

    def __init__(self, data_fn, input_vars, output_vars,
             norm_fn=None, input_transform=None, output_transform=None,
             batch_size=1024, shuffle=True, xarray=False, var_cut_off=None, normalize_flag=True,
             rh_trans=True,t2tns_trans=True,
             lhflx_trans=True,
             scaling=True,interpolate=True,
             hyam=None,hybm=None,
             inp_subRH=None,inp_divRH=None,
             inp_subTNS=None,inp_divTNS=None,
             lev=None, interm_size=40,
             lower_lim=6,
             is_continous=True,Tnot=5,
                mode='train', exp=None):
        self.scaling = scaling
        self.interpolate = interpolate
        self.rh_trans = rh_trans
        self.t2tns_trans = t2tns_trans
        self.lhflx_trans = lhflx_trans
        self.inp_shape = 64
        self.exp = exp
        self.mode=mode
        super().__init__(data_fn, input_vars,output_vars,norm_fn,input_transform,output_transform,
                        batch_size,shuffle,xarray,var_cut_off,normalize_flag) ## call the base data generator
        self.inp_sub = self.input_transform.sub
        self.inp_div = self.input_transform.div
        self.new_idx = np.concatenate((np.arange(8,26),np.arange(34,52),np.arange(60,78),np.arange(86,104),np.arange(104,108)))
        self.new_output_idx = np.concatenate((np.arange(8,26),np.arange(26,52),np.arange(60,78),np.arange(86,104),np.arange(104,112)))

        if self.rh_trans:
            self.qv2rhLayer = QV2RHNumpyReal(self.inp_sub,self.inp_div,inp_subRH,inp_divRH,hyam,hybm)

    def __getitem__(self, index):
        # Compute start and end indices for batch
        start_idx = index * self.batch_size
        end_idx = start_idx + self.batch_size
        # Grab batch from data
        batch = self.data_ds['vars'][start_idx:end_idx]
        X = batch[:, self.input_idxs]
        Y = batch[:, self.output_idxs]
        # Normalize
        X_norm = self.input_transform.transform(X)
        Y = self.output_transform.transform(Y)
        return X_norm[:,self.new_idx], Y

In [30]:
in_vars_RH = ['RH','TBP','CLDLIQBP','CLDICEBP','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars_RH =['QBCTEND','TBCTEND','CLDLIQBCTEND', 'CLDICEBCTEND', 'NN2L_FLWDS', 'NN2L_PRECC', 
            'NN2L_PRECSC', 'NN2L_SOLL', 'NN2L_SOLLD', 'NN2L_SOLS', 'NN2L_SOLSD', 'NN2L_NETSW']

TRAINFILE_RH = 'RG_RH_M4K_NORM_train_shuffle.nc'
NORMFILE_RH = 'RG_RH_M4K_NORM_norm.nc'
VALIDFILE_RH = 'RG_RH_M4K_NORM_valid.nc'
BASE_DIR = '/DFS-L/DATA/pritchard/ankitesg/'

In [31]:
train_gen_RH = DataGenerator(
    data_fn = f"{BASE_DIR}datav4/{TRAINFILE_RH}",
    input_vars = in_vars_RH,
    output_vars = out_vars_RH,
    norm_fn = f"{BASE_DIR}datav4/{NORMFILE_RH}",
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True,
    normalize_flag=True
)

In [32]:
in_vars = ['QBP','TBP','CLDLIQBP','CLDICEBP','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars = ['QBCTEND','TBCTEND','CLDLIQBCTEND', 'CLDICEBCTEND', 'NN2L_FLWDS', 'NN2L_PRECC', 
            'NN2L_PRECSC', 'NN2L_SOLL', 'NN2L_SOLLD', 'NN2L_SOLS', 'NN2L_SOLSD', 'NN2L_NETSW']

In [33]:
TRAINFILE = 'RG_SP_M4K_train_shuffle.nc'
NORMFILE = 'RG_SP_M4K_NORM_norm.nc'
VALIDFILE = 'RG_SP_M4K_valid.nc'

In [34]:
train_gen = DataGeneratorClimInvRealGeo(
    data_fn = f'{DATA_DIR}{TRAINFILE}',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = f'{DATA_DIR}{NORMFILE}',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True,
    normalize_flag=True,
    lev=lev,
    hyam=hyam,hybm=hybm,
    rh_trans = True,t2tns_trans=False,
    lhflx_trans=False,
    scaling=False,
    interpolate=False,
    inp_subRH=train_gen_RH.input_transform.sub,inp_divRH=train_gen_RH.input_transform.div
)


In [35]:
valid_gen = DataGeneratorClimInvRealGeo(
    data_fn = f'{DATA_DIR}{VALIDFILE}',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = f'{DATA_DIR}{NORMFILE}',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True,
    normalize_flag=True,
    lev=lev,
    hyam=hyam,hybm=hybm,
    rh_trans = True,t2tns_trans=False,
    lhflx_trans=False,
    scaling=False,
    interpolate=False,
    inp_subRH=train_gen_RH.input_transform.sub,inp_divRH=train_gen_RH.input_transform.div
)


In [36]:
model = Sequential()
model.add(Input(shape=(76,)))
model.add(Dense(128, activation='linear'))
model.add(LeakyReLU(alpha=0.3))
for i in range (4):
    model.add(Dense(128, activation='linear'))
    model.add(LeakyReLU(alpha=0.3))
    
for i in range (3):
    model.add(Dense(256, activation='linear'))
    model.add(LeakyReLU(alpha=0.3))
    
model.add(Dense(112, activation='linear'))

In [37]:
opt = tf.keras.optimizers.Adam()

In [38]:
model.compile(optimizer=opt, loss='mse')

In [39]:
# model.compile(tf.keras.optimizers.Adam(), loss="mse")
path_HDF5 = '/DFS-L/DATA/pritchard/ankitesg/models/'
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint(path_HDF5+'RH_RG_TrimV1.h5',save_best_only=True, monitor='val_loss', mode='min')

In [40]:
with tf.device('/gpu:0'):
    Nep = 15
    model.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen,\
                  callbacks=[earlyStopping, mcp_save])

Epoch 1/15
42120/42120 [==============================] - 460s 11ms/step - batch: 21059.5000 - size: 1024.0000 - loss: 225.8808 - val_loss: 200.8910
Epoch 2/15
42120/42120 [==============================] - 462s 11ms/step - batch: 21059.5000 - size: 1024.0000 - loss: 189.1600 - val_loss: 187.4659
Epoch 3/15
42120/42120 [==============================] - 461s 11ms/step - batch: 21059.5000 - size: 1024.0000 - loss: 181.9264 - val_loss: 182.6423
Epoch 4/15
42120/42120 [==============================] - 453s 11ms/step - batch: 21059.5000 - size: 1024.0000 - loss: 178.0732 - val_loss: 177.9773
Epoch 5/15
42120/42120 [==============================] - 456s 11ms/step - batch: 21059.5000 - size: 1024.0000 - loss: 175.6647 - val_loss: 178.2746
Epoch 6/15
42120/42120 [==============================] - 456s 11ms/step - batch: 21059.5000 - size: 1024.0000 - loss: 173.7339 - val_loss: 176.8120
Epoch 7/15
42120/42120 [==============================] - 454s 11ms/step - batch: 21059.5000 - size: 1024.

### CLDLIQ and CLDICE Trimmed

In [41]:
class DataGeneratorClimInvRealGeo(DataGenerator):

    def __init__(self, data_fn, input_vars, output_vars,
             norm_fn=None, input_transform=None, output_transform=None,
             batch_size=1024, shuffle=True, xarray=False, var_cut_off=None, normalize_flag=True,
             rh_trans=True,t2tns_trans=True,
             lhflx_trans=True,
             scaling=True,interpolate=True,
             hyam=None,hybm=None,
             inp_subRH=None,inp_divRH=None,
             inp_subTNS=None,inp_divTNS=None,
             lev=None, interm_size=40,
             lower_lim=6,
             is_continous=True,Tnot=5,
                mode='train', exp=None):
        self.scaling = scaling
        self.interpolate = interpolate
        self.rh_trans = rh_trans
        self.t2tns_trans = t2tns_trans
        self.lhflx_trans = lhflx_trans
        self.inp_shape = 64
        self.exp = exp
        self.mode=mode
        super().__init__(data_fn, input_vars,output_vars,norm_fn,input_transform,output_transform,
                        batch_size,shuffle,xarray,var_cut_off,normalize_flag) ## call the base data generator
        self.inp_sub = self.input_transform.sub
        self.inp_div = self.input_transform.div
        self.new_idx = np.concatenate((self.input_idxs[8:26],self.input_idxs[34:52],self.input_idxs[60:78],self.input_idxs[86:104],\
        self.input_idxs[104:]))
        self.new_idx = np.concatenate((np.arange(8,26),np.arange(34,52),np.arange(60,78),np.arange(86,104),np.arange(104,108)))


    def __getitem__(self, index):
        # Compute start and end indices for batch
        start_idx = index * self.batch_size
        end_idx = start_idx + self.batch_size

        # Grab batch from data
        batch = self.data_ds['vars'][start_idx:end_idx]
#         print(self.new_idx)
        # Split into inputs and outputs
        X = batch[:, self.input_idxs]
        Y = batch[:, self.output_idxs]
        # Normalize
        X_norm = self.input_transform.transform(X)
        Y = self.output_transform.transform(Y)
        return X_norm, Y

In [42]:
in_vars = ['QBP','TBP','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars = ['QBCTEND','TBCTEND','CLDLIQBCTEND', 'CLDICEBCTEND', 'NN2L_FLWDS', 'NN2L_PRECC', 
            'NN2L_PRECSC', 'NN2L_SOLL', 'NN2L_SOLLD', 'NN2L_SOLS', 'NN2L_SOLSD', 'NN2L_NETSW']

In [43]:
TRAINFILE = 'RG_SP_M4K_train_shuffle.nc'
NORMFILE = 'RG_SP_M4K_NORM_norm.nc'
VALIDFILE = 'RG_SP_M4K_valid.nc'

In [45]:
train_gen = DataGeneratorClimInvRealGeo(
    data_fn = f'{DATA_DIR}{TRAINFILE}',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = f'{DATA_DIR}{NORMFILE}',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True,
    normalize_flag=True,
    lev=lev,
    hyam=hyam,hybm=hybm,
    rh_trans = False,t2tns_trans=False,
    lhflx_trans=False,
    scaling=False,
    interpolate=False,
    inp_subRH=train_gen_RH.input_transform.sub,inp_divRH=train_gen_RH.input_transform.div
)


In [46]:
valid_gen = DataGeneratorClimInvRealGeo(
    data_fn = f'{DATA_DIR}{VALIDFILE}',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = f'{DATA_DIR}{NORMFILE}',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True,
    normalize_flag=True,
    lev=lev,
    hyam=hyam,hybm=hybm,
    rh_trans = False,t2tns_trans=False,
    lhflx_trans=False,
    scaling=False,
    interpolate=False,
    inp_subRH=train_gen_RH.input_transform.sub,inp_divRH=train_gen_RH.input_transform.div
)


In [51]:
model = Sequential()
model.add(Input(shape=(56,)))
model.add(Dense(128, activation='linear'))
model.add(LeakyReLU(alpha=0.3))
for i in range (4):
    model.add(Dense(128, activation='linear'))
    model.add(LeakyReLU(alpha=0.3))
    
for i in range (3):
    model.add(Dense(256, activation='linear'))
    model.add(LeakyReLU(alpha=0.3))
    
model.add(Dense(112, activation='linear'))

In [52]:
opt = tf.keras.optimizers.Adam()

In [53]:
model.compile(optimizer=opt, loss='mse')

In [54]:
# model.compile(tf.keras.optimizers.Adam(), loss="mse")
path_HDF5 = '/DFS-L/DATA/pritchard/ankitesg/models/'
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint(path_HDF5+'BF_RG_ReducedInpV1.h5',save_best_only=True, monitor='val_loss', mode='min')

In [55]:
with tf.device('/gpu:0'):
    Nep = 15
    model.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen,\
                  callbacks=[earlyStopping, mcp_save])

Epoch 1/15
42120/42120 [==============================] - 485s 12ms/step - batch: 21059.5000 - size: 1024.0000 - loss: 341.1907 - val_loss: 304.9829
Epoch 2/15
42120/42120 [==============================] - 643s 15ms/step - batch: 21059.5000 - size: 1024.0000 - loss: 289.8065 - val_loss: 287.2073
Epoch 3/15
42120/42120 [==============================] - 421s 10ms/step - batch: 21059.5000 - size: 1024.0000 - loss: 279.2112 - val_loss: 284.6332
Epoch 4/15
42120/42120 [==============================] - 413s 10ms/step - batch: 21059.5000 - size: 1024.0000 - loss: 273.5045 - val_loss: 281.2119
Epoch 5/15
42120/42120 [==============================] - 417s 10ms/step - batch: 21059.5000 - size: 1024.0000 - loss: 269.7371 - val_loss: 286.4447
Epoch 6/15
42120/42120 [==============================] - 407s 10ms/step - batch: 21059.5000 - size: 1024.0000 - loss: 266.7937 - val_loss: 286.8031
Epoch 7/15
42120/42120 [==============================] - 416s 10ms/step - batch: 21059.5000 - size: 1024.